In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests as rq
import numpy as np
import time
import pickle
from io import BytesIO
from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from collections import Counter
import cv2
import itertools
import seaborn as sns
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package wordnet to /home/pidoux/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Scrap des images pour un hashtag

In [2]:
USERNAME = 'sosorezka@gmail.com'
PASSWORD = 'sofia2001'

In [3]:
def scrap_by_hashtag(nb_max = 50, hashtag='chat'): 

    options = webdriver.ChromeOptions()
    options.headless = True  # Exécution sans interface graphique

    # Initialiser le navigateur Chrome sans spécifier executable_path
    driver = webdriver.Chrome(options=options)

    # Accéder à la page de connexion de Pixelfed
    login_url = 'https://pixelfed.social/login'
    driver.get(login_url)

    # Remplir le formulaire de connexion
    username = USERNAME
    password =  PASSWORD

    username_field = driver.find_element(By.NAME, 'email')
    password_field = driver.find_element(By.NAME, 'password')

    username_field.send_keys(username)
    password_field.send_keys(password)

    # Soumettre le formulaire de connexion
    login_button = driver.find_element(By.CSS_SELECTOR, "button.btn.btn-primary.btn-block.btn-lg.font-weight-bold")
    ActionChains(driver).move_to_element(login_button).click().perform()

    driver.implicitly_wait(10)
    
    # Ouvrir une nouvelle fenêtre
    new_window = driver.execute_script("window.open('https://pixelfed.social/i/web/hashtag/{}', 'new_window');".format(hashtag))

    # Changer la fenêtre active
    driver.switch_to.window(driver.window_handles[-1])

    liste_url = []
    liste_img = []
    pbar = tqdm(desc= 'while_loop', total=nb_max)
    while len(liste_url)<nb_max :
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        time.sleep(1)
        print(len(liste_url))
        post =driver.find_element(By.CSS_SELECTOR, "div.row.mx-0.hashtag-feed")
        #print(post)
        # Trouver les images
        images = post.find_elements(By.CSS_SELECTOR, "img")
        val = 0
        for image in tqdm(images) :
            image_url = image.get_attribute('src')
            if image_url not in liste_url : 
                try : 
                    response = rq.get(image_url)
                    img = np.array(Image.open(BytesIO(response.content)))
                    if img.shape[2]==3 and img.shape[0]>512 and img.shape[1]>512: #si l'image est en couleur et a une shape plus grande que 128x128
                        liste_url.append(image_url)
                        liste_img.append(img)
                        print(len(liste_img))
                        val+=1
                        pbar.update(val)
                except:
                    print("erreur get")
            
    df = pd.DataFrame({'url':liste_url, 'image':liste_img})
    print(len(liste_url))
    
    # Fermer la fenêtre
    driver.close()

    # Récupérer la fenêtre principale
    driver.switch_to.window(driver.window_handles[0])

    # Fermer le navigateur
    driver.quit()

    with open(f'dataframe_{hashtag}.pkl', 'wb') as file : 
        pickle.dump(df, file)
    return df

In [5]:
test = scrap_by_hashtag(nb_max=200, hashtag='cat')

# Test de l'enregistrement du pickle 

In [11]:
with open('dataframe_cat.pkl', 'rb') as file : 
    df = pickle.load(file)
df.head(2)